## Supporting Information.
Borofsky, T. and Feldman, M. 2020. Static environments with limited resources select for multiple foraging strategies rather than conformity in social learners.
# Appendix S2: Solving for Initial Equilibria

In [2]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import helperfuns
from helperfuns import *
import DatFrameFuns
from DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
#import sympy as sp
#from sympy import *
#from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool




## First we set up a parameter mesh, as described in Section 5.1

In [3]:

Dvals = np.sort(np.arange(-2,1.2,0.2))
svals = np.sort(np.arange(0,4.5,0.5))
muvals = np.array([-2, -1, -0.5, -0, 0, 0.5,1,2])
betavals = np.arange(0,1.25,0.25)
df = get_param_grid(Dvals, svals, muvals, betavals)

## We use parallelization to to greatly speed up the dataframe generation. 

In [3]:

# Parallelize and find equilibria

cores=mp.cpu_count()
# split dataframe over cores, stored in df_split
df_split = np.array_split(df, cores, axis=0)
pool = Pool(cores)

# Run get_1side_df over each dataframe in df_split. 
# get_1side_df iterates 50000 time steps. Stack dataframe (df) parts into one df.

df_out = np.vstack(pool.map(get_1side_df, df_split))
pool.close()
pool.join()
pool.clear()
df_1side = pd.DataFrame(df_out, columns = df.columns)

# fix the column names
colnames = ['mu', 'K', 'pc', 's', 'D', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df_1side = df_1side[colnames] # gets rid of the added unnamed columns
#save to csv
df_1side.to_csv('data_1side_1stRound.csv')

## Check the result of the 200,000th iteration to see if it is at equilibrium.

In [45]:
df_1side = pd.read_csv('data_1side_1stRound.csv')

In [46]:
# Set error tolerance levels for when I check if the equilibrium fits the equilibrium equation
rtol = 1e-10
atol = 1e-10

# shortening variable names for readability
u = 1
Weq = df_1side.Weq
K = df_1side.K
D = df_1side.D
pc = df_1side.pc
bu = df_1side.bueq
u1 = df_1side.u1eq
u2 = df_1side.u2eq
r1 = df_1side.r1eq
r2 = df_1side.r2eq

In [47]:
# These are the recursion systems when u = 1 (only AB foragers) at equilibrium
should0_u1 = Weq*u1 - u*(K*(u1 + D*phi_fun(u1)) + pc)*(1+r1)
should0_u2 = Weq*u2 - u*(K*(u2 + D*phi_fun(u2)) + pc)*(1+r2)
should0_bu = Weq*bu - u*(K*(1 - u1 - u2 - D*phi_fun(u1) - D*phi_fun(u2)) + 1-K - pc)

reached_eq = np.isclose(should0_u1,0,rtol=rtol, atol = atol)
reached_eq = reached_eq & np.isclose(should0_u2,0,rtol=rtol, atol = atol)
reached_eq = reached_eq & np.isclose(should0_bu,0,rtol=rtol, atol = atol)

df_1side.reached_eq = reached_eq

In [49]:
# separate those that reached equilibrium from those that did not
df_correct = df_1side.query('reached_eq==True')
df_fail = df_1side.query('reached_eq==False')
len(df_fail)

255

### We see that 255 did not reach equilibrium in 200000 steps. For each of these, $\beta = 0$, $D = 0$, and $\pi_{C} < 1 \times 10^{-5}$ but $\pi_{C} \neq 0$. Also, for these $K > 1/2$ (shown in the cells below). We showed in S4.3 that equilibria with these parameters are internally unstable.

In [26]:
np.unique(df_fail.beta)

array([0.])

In [30]:
np.unique(df_fail.s)
np.unique(df_fail.K)

array([0.69145906, 0.84134446, 0.93319278, 0.9772182 , 0.97724987,
       0.99378694, 0.99861843, 0.99864982, 0.99976397, 0.99993666])

In [28]:
np.unique(df_fail.D)

array([-4.4408921e-16])

In [29]:
np.unique(df_fail.pc)

array([9.86587700e-10, 1.89895625e-08, 2.86651572e-07, 3.39767312e-06,
       3.16712418e-05])

## Now try to fill in rows that didn't reach equilibrium

In [53]:
df_fail.time = -1 # so I know these were not iterated to equilibrium and thus are not internally stable.
# from scipy.optimize import fsolve
df_fixed = fsolve_failed_eq(df_fail)

df_1side = df_correct.append(df_fixed)
#df_1side.to_csv('data_1side_fixed.csv')
df_1side.to_csv('data_1side_fixed.csv')

# reflect equilibria in the manner described in the Numerical Analysis, equations 24 - 25.
df_total = reflect_df(df_1side)

# make it look good.
df = df_total.sort_values(by=['mu','beta','s','D'])
df.reset_index(inplace=True, drop=True) 
df = df[colnames]
# There's an issue with setting D = 0. I'm getting weird python e-16 values instead. 
# This can make code difficult later on, so I'm rounding it now
df.D = df['D'].round(6)
# We want later to be able to count how many times iteration failed for each equilibrium
df['iterated'] = df.time!= -1
df.to_csv('data.csv')

/share/software/user/open/py-pandas/0.23.0_py36/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
def get_UniqueEquilibria(df,if_save=False):
    df_eq = df.round(6)[(df.reached_eq==1)&(df.iterated==1)].groupby(['K','pc','s','mu','D','beta','u1eq','u2eq','bueq',
                                                     'r1eq','r2eq','Weq','URstable'], as_index = False)
    df_eq = df_eq['u2init'].count()
    df_eq.rename(columns={'u2init':'NumInitials'}, inplace=True)
    # df_eq.reset_index(inplace=True, drop=True)
    
    df_eq = df_eq.apply(lambda row: JstarStable(row), axis = 1) # This step was removed because 
                                                                  # we are only working with equilibria 
                                                                  # which were reached through iteration
    df_eq = get_gradients(df_eq)
    if if_save:
        df_eq.to_csv('UniqueEquilibria.csv', index = False)
    return(df_eq)

In [5]:
# We have a lot of repeat equilibria in the dataframe df because many of the initial points go to the same equilibrium.
# We now extract the unique equilibria for each parameter combination.
df = pd.read_csv('data.csv')
df_main = df[(df.reached_eq==1)&(df.iterated==1)]
unique_eq = get_UniqueEquilibria(df_main)
diff = np.abs(unique_eq.u1eq - unique_eq.u2eq)
unique_eq['difference'] = diff
unique_eq.to_csv('UniqueEquilibriaDF.csv')

# Check Stability

It helps to have columns documenting which alleles can invade. These columns are x_pos_invades, x_neg_invades, D_pos_invades, and D_neg_invades:
- x_pos_invades = TRUE if $C_s > 0$ and $K < 1$. Else false
- x_neg_invades = TRUE if $C_s < 0$ and $K > 0$. Else false
- D_pos_invades = TRUE if $C_D > 0$ and $D < 1$. Else false
- D_neg_invades = TRUE if $C_D < 0$ and $D > -2$  
There are many cases of $C_D = 0$ that need to be resolved. There are no cases of $C_s = 0$.

In [7]:
df_check = unique_eq.sample(10)
df_out = df_ext_stability_iterate(df_check)
cores=mp.cpu_count()
df_split = np.array_split(df_check, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(df_ext_stability_iterate, df_split))
pool.close()
pool.join()
pool.clear()
cols =  df_check.columns
df_fixed = pd.DataFrame(df_out, columns = cols)

NameError: name 'cols' is not defined

In [9]:
cols =  df_check.columns
df_fixed = pd.DataFrame(df_out, columns = cols)
df_fixed

,K,pc,s,mu,D,beta,u1eq,u2eq,bueq,r1eq,...,Weq,URstable,NumInitials,C_s,C_D,difference,x_pos_invades,x_neg_invades,y_neg_invades,y_pos_invades
0,0.499968,0.5,2.0,2.0,-0.6,0.75,0.499992,0.499992,0.000015,0.625006,...,2.437485,1.0,64.0,-0.265931,-0.000001,0.0,False,True,True,True
1,0.933193,0.0,3.5,-2.0,0.6,0.75,0.0,0.0,1.0,1.0,...,1.0,1.0,16.0,-0.0,-0.0,0.0,True,True,True,False
2,0.999937,0.000032,4.0,0.0,-1.2,1.0,0.499986,0.499986,0.000029,0.500014,...,1.50004,1.0,128.0,-0.000089,-0.000005,0.0,False,True,True,False
3,0.977218,0.000032,3.0,-1.0,-1.4,0.0,0.493229,0.493229,0.013541,1.0,...,1.973342,1.0,64.0,0.027113,-0.003352,0.0,True,False,True,False
4,0.9545,0.02275,2.0,0.0,1.0,0.5,0.058325,0.058325,0.883351,0.970838,...,1.085102,1.0,32.0,-0.144473,-0.082865,0.0,False,True,True,False
5,0.926983,0.066807,2.0,0.5,0.4,0.75,0.494945,0.494945,0.010109,0.628791,...,1.726631,1.0,64.0,-0.116445,-0.001706,0.0,False,True,True,False
6,0.9759,0.02275,2.5,0.5,0.4,1.0,0.498136,0.498136,0.003728,0.501864,...,1.533163,1.0,64.0,-0.051524,-0.000595,0.0,False,True,True,False
7,0.47725,0.5,1.0,1.0,-0.2,1.0,0.493735,0.493735,0.01253,0.506265,...,2.245155,1.0,64.0,-0.256626,-0.000674,0.0,False,True,True,False
8,0.993558,0.000233,3.0,-0.5,-0.4,0.0,0.497402,0.497402,0.005195,1.0,...,1.990126,1.0,64.0,0.007892,-0.001297,0.0,True,False,True,False
9,0.99865,0.0,4.0,-1.0,0.2,0.5,0.49896,0.49896,0.002079,0.75052,...,1.747793,1.0,64.0,0.001897,-0.000446,0.0,True,False,True,False


In [32]:
#df = pd.read_csv('UniqueEquilibriaDF.csv')
df = unique_eq
#df = df.drop(columns='Unnamed: 0')

y_neg_invades = df.C_D < 0
df['y_neg_invades'] = y_neg_invades
y_pos_invades = df.C_D > 0
df['y_pos_invades'] = y_pos_invades
x_neg_invades = df.C_s < 0
df['x_neg_invades']=x_neg_invades
x_pos_invades = df.C_s > 0
df['x_pos_invades'] = x_pos_invades

# account for border issues
mask_K0_change = (df['C_s'] < 0).values & (df['s'] == 0).values

df.loc[mask_K0_change, ['x_neg_invades']] = False
mask_Dlower_change = (df.C_D < 0).values & (df.D == -2).values
df.loc[mask_Dlower_change, ['y_neg_invades']] = False
mask_Dupper_change = (df.C_D > 0).values & (df.D == 1).values
df.loc[mask_Dupper_change, ['y_pos_invades']] = False




In [13]:

cols = df.columns
df_normal = df.query('C_D!=0')
df_weird = df.query('C_D==0')


In [2]:
unique_eq = pd.read_csv('UniqueEquilibriaDF.csv')
df_weird = unique_eq.head()
cores=mp.cpu_count()
df_split = np.array_split(df_weird, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(df_ext_stability_iterate, df_split))
pool.close()
pool.join()
pool.clear()
df_fixed = pd.DataFrame(df_out, columns = cols) # restoring original column names
#df_total = df_normal.append(df_fixed)
#df_total.to_csv('UniqueEquilibriaDF.csv')

KeyError: "None of [Index(['s', 'D', 'mu', 'beta', 'K', 'pc'], dtype='object')] are in the [index]"

In [ ]:
''' A better way to check stability'''

cores=mp.cpu_count()
df_split = np.array_split(df_weird, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(get_stable_check, df_split))
pool.close()
pool.join()
pool.clear()
df_fixed = pd.DataFrame(df_out, columns = cols) # restoring original column names
df_total = df_normal.append(df_fixed)
df_total.to_csv('UniqueEquilibriaDF.csv')

# Next, go to SensitivityAnalysis.Rmd to see analysis of these equilibria

No longer used:

In [1]:

def Check_Stable_D(row):
# We have a bunch of points with C_D = 0. Check_Stable_D checks if they're actually stable

    row = row.copy() # explicitly tell python i'm making a copy so i don't get the warning
    
    # get vectors of post perturb values of all the frequencies
    umat,ymat,rmat = Perturb(row) # Perturb is a function defined below
    umat = np.array(umat); ymat = np.array(ymat); rmat = np.array(rmat)
    # check stability to increase in D
    
    dD =  0.01
    n = len(umat[0]) 
    y_pos_invades = False # default is that it is stable
    if row.D <= 0.99: # because if D >= 0.99 then dD \geq 0.01 can't invade
        for i in range(0,n):
            result = NextGen(umat[:,i],[0,0,0],ymat[:,i],rmat[:,i], 
                             row.D, row.K,row.pc,row.beta,
                             deltas = [dD, 0, 0], eta=1)
            yvec = result[2]
            y = sum(yvec)
            if y > 0.01: # asking: after 1 iteration, does y increase?
                y_pos_invades = True
                break
    # We don't need to worry about cases of 0.99 < D < 1 because I don't have any of those
    
    dD = -0.01
    y_neg_invades = False # default is that it is stable
    #check stability to decrease in D
    if row.D >= -1.99: # otherwise it's stable because D can't go down further
        for i in range(0,n):
            result = NextGen(umat[:,i],[0,0,0],ymat[:,i],rmat[:,i], 
                             row.D, row.K,row.pc,row.beta,
                             deltas = [dD, 0, 0], eta=1)
            yvec = result[2]
            y = sum(yvec)
            if y > 0.01: 
                y_neg_invades = True 
                break
    
    row.loc['y_pos_invades'] = y_pos_invades
    row.loc['y_neg_invades'] = y_neg_invades
    return(row)

def Perturb(row):
# After the [u1,u2,bu,r1,r2] eq is perturbed with the addition of the a or b allele, get new frequencies
# perturb by a magnitude of 0.01... so |dr1| = |dr2| = |du| = 0.01, and either |dx| or |dy| = 0.01

    # use dyvec here but dxvec could work too
    uvec = [row.u1eq, row.u2eq, row.bueq]
    rvec = [row.r1eq, row.r2eq]
    # get new post-perturb vectors
    
    # no need to check if valid
    dy = 0.01
    dy1vec = np.array([0.05, 0.3,0.48, 0.08])*dy
    dy2vec = np.array([0.1,0.6,0.9])*dy
    dbyvec = dy - dy1vec - dy2vec
    which_y = [0,1,2,3]
    
    #dr... prune r + dr values that are invalid
    
    dr1=0 #the default is not changing r1
    if rvec[0]>0:
        dr1 = np.array([-0.01,0.01])
        check_r1 = (rvec[0] + dr1 > 0)&(rvec[0] + dr1 < 1)
        dr1 = dr1[check_r1]
    dr2 = 0
    if rvec[1]>0:
        dr2 = np.array([-0.01,0.01])
        check_r2 = (rvec[1] + dr2 > 0) & (rvec[1] + dr2 < 1)
        dr2 = dr2[check_r2]
    
    # du perturbations... look similar to x perturbs, but opposite direction
    du = -dy
    du1vec = np.array([0.015, 0.29,0.49, 0.07])*du
    du2vec = np.array([0.11,0.49,0.51,0.89])*du
    dbuvec = du - du1vec - du2vec
    which_u = [0,1,2,3]
    for i in which_u:
        duvec = [du1vec[i],du2vec[i],dbuvec[i]]
        [du1vec[i],du2vec[i],dbuvec[i]] = Perturb_EdgeCase(uvec,duvec)
    
    # now find all the combinations
    which_y_mesh, which_u_mesh, DR1, DR2 = np.meshgrid(which_y, which_u, dr1, dr2)
    [which_y, which_u, dr1, dr2] = [np.ndarray.flatten(item) for 
                                    item in [which_y_mesh, which_u_mesh, DR1, DR2]]
    y1 = dy1vec[which_y]; y2 = dy2vec[which_y]; by = dbyvec[which_y]
    u1 = uvec[0] + du1vec[which_u]
    u2 = uvec[1] + du2vec[which_u]
    bu = uvec[2] + dbuvec[which_u]
    r1 = rvec[0] + dr1
    r2 = rvec[1] + dr2
    
    
    return([u1,u2,bu],[y1,y2,by],[r1,r2])

def Perturb_EdgeCase(uvec,duvec):
    #recursively checks for edge cases so i don't get an invalid frequency. Adjusts duvec if needed
    
    # make sure using numpy arrays
    du = sum(duvec)
    uvec = np.array(uvec); duvec = np.array(duvec);
    # find locations of edge cases
    edge_bool = uvec + duvec <= 0
    
    n = sum(edge_bool)
    if n>0:
        duvec[edge_bool] = -uvec[edge_bool] +0.00001 # so not at exactly 0
        du_remain = du - sum(duvec)
        duvec[~edge_bool] = duvec[~edge_bool] + (1/np.float64(3-n))*du_remain
        
        # make sure that we didn't cause a different frequency to be negative:
        return(Perturb_EdgeCase(uvec,duvec))

    else:
        return(duvec)


def get_stable_check(df):
    # this function is just a wrapper to be used with pool.map that takes a dataframe as an input and
    # runs Check_Stable_D over the data frame
    df = df.apply(lambda row: Check_Stable_D(row), axis = 1)
    return(df)

SyntaxError: invalid syntax (<ipython-input-1-3619a1b5057e>, line 55)